<a href="https://colab.research.google.com/github/mohammadshakournia/Data-Science-Projects/blob/master/CTR%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip -q "/content/drive/My Drive/datadays2020_contest_public_dataset.zip"

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

ad=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/ad.csv')
ad_image=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/ad_image.csv')
click_train=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/click_train.csv')
event=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/event.csv')
click_test=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/click_test.csv')
ad_title=pd.read_csv('/content/drive/My Drive/datadays2020_contest_public_dataset.zip (Unzipped Files)/datadays2020_contest_public_dataset/ad_title.csv')


## **Preprocessing**

Extract image feature of each ad to seprated columns

In [0]:

ad_image1=ad_image['imageFeatures'].values.tolist()

for i in range(len(ad_image1)):
    ad_image1[i]=ad_image1[i][1:len(ad_image1[i])-1]
    ad_image1[i]=ad_image1[i].split(', ')

ad_image1=[[float(i) for i in x] for x in ad_image1]

dd=pd.DataFrame(ad_image1)
dd['adId']=ad_image['adId']

cols=dd.columns
cols = ['adId']  + [col for col in dd if col != 'adId']
ad_image = dd[cols]
del dd,cols,ad_image1,i


Merge adId and ad_image for making a new complete dataframe for ad informations 

In [0]:
ad_info=pd.merge(ad,ad_image,how='inner',on='adId')

Calculate count of clicked ad from click_train dataset and divide them on whole view number of ads to take percentage of clicked ads.
and remove ads with percentage less than the mean of all pecentages

In [0]:
main=click_train[click_train['clicked']!=0]
main=main.drop(["displayId",'clicked'],axis=1)
main["clicked_number"] = main.groupby(["adId"])["adId"].transform("count")
clicked_ads=main.drop_duplicates()

main2=pd.DataFrame(columns=['adId'])
main2['adId']=click_train['adId']
main2["total_viewed_number"] = main2.groupby(["adId"])["adId"].transform("count")
main2=main2.drop_duplicates()

main_final=pd.merge(main2,clicked_ads,how='left',on=["adId","adId"])
main_final=main_final.fillna(0)
main_final['Percentage']=(main_final['clicked_number']/main_final['total_viewed_number'])*100
clicked_ads_informations=pd.merge(click_train,main_final,how='left',on=['adId','adId'])

del main,main2,clicked_ads,main_final

clicked_ads_informations=clicked_ads_informations[clicked_ads_informations['Percentage']>(clicked_ads_informations['Percentage'].mean())]


final=pd.merge(clicked_ads_informations,event,how='left',on='displayId')
cols=final.columns.tolist()

cols.remove("total_viewed_number")
cols.remove('clicked_number')
cols.remove("Percentage")

cols.append("total_viewed_number")
cols.append('clicked_number')
cols.append("Percentage")

final=final[cols]



merge click_test with even dataset because we need same kind of dataset for tarin and test datasets.
remove timestamp column because this just shows how many times a user views a page

In [0]:
click_test=pd.merge(click_test,event,how='left',on='displayId')
click_test=click_test.drop(columns=['timestamp'])

remove useless variable and columns

In [0]:
final=final.drop(columns=['timestamp','Percentage'])

In [0]:
final=final.drop(columns=['total_viewed_number','clicked_number'])

In [0]:
df=pd.merge(final,ad_image,how='inner',on='adId')

In [0]:
click_test=pd.merge(click_test,ad_image,how='inner',on='adId')

In [0]:
ad=0
ad_image=0
ad_info=0
click_train=0
clicked_ads_informations=0
cols=0
final=0
event=0

one hot technique for extract word id of each ad

In [0]:
ad_title=ad_title.groupby('adId')['wordId'].apply(list)
ad_title=ad_title.to_frame()
ad_title.reset_index(level=0, inplace=True)
ad_title['wordId'] =ad_title['wordId'].apply(lambda x : [int(i) for i in x])

x=pd.get_dummies(ad_title['wordId'].apply(pd.Series).stack()).sum(level=0)
x['adId']=ad_title['adId']

cols=x.columns
cols = ['adId']  + [col for col in x if col != 'adId']
ad_title = x[cols]
del x, cols


In [0]:
a=0
ad_title=[]

In [0]:
ad=0
ad_image=0
ad_info=0
click_train=0
event=0
final=0
ad_title=0
clicked_ads_informations=0

In [0]:
y=df.clicked
df=df.drop(columns=['clicked'])

In [0]:
import sys
sys.getsizeof(df)

6810253608

## **Train and Predict**

train with decision tree

In [0]:
clf = DecisionTreeClassifier()
clf = clf.fit(df,y)

In [0]:
y_pred = clf.predict(click_test)

make result dataset

In [0]:
resualt=pd.DataFrame()
resualt['displayId']=click_test['displayId']
resualt['adId']=click_test['adId']
resualt['click_prediction']=y_pred


based on clicked times for each ad we ranked it

In [0]:
resualt['clicked_times_per_ad'] = resualt.groupby(["adId"])["click_prediction"].transform("count")


In [0]:
resualt

,displayId,adId,click_prediction,clicked_times_per_ad
0,151650,7585,0,5941
1,240394,7585,0,5941
2,244567,7585,0,5941
3,244664,7585,0,5941
4,244777,7585,0,5941
...,...,...,...,...
1199043,1327627,7168,0,11
1199044,1327877,7168,0,11
1199045,1328140,7168,0,11
1199046,1328722,7168,0,11


sort and make a rank column for result

In [0]:
resualt=resualt.sort_values(by=['displayId','clicked_times_per_ad'], ascending=False)

In [0]:
resualt

,displayId,adId,click_prediction,clicked_times_per_ad
693743,1328976,535,1,17501
807891,1328976,7182,0,11720
498028,1328976,7174,1,4335
855691,1328976,7167,0,944
123692,1328957,6515,0,41874
...,...,...,...,...
12150,151938,7715,0,387
8907,151938,7454,0,317
0,151650,7585,0,5941
7056,151650,6690,1,1851


In [0]:
z=resualt.click_prediction.eq(1).groupby(resualt.adId).sum().astype(int).reset_index()
z

,adId,click_prediction
0,8,806
1,11,265
2,14,48
3,17,7567
4,19,601
...,...,...
1345,7862,2
1346,7863,1
1347,7865,1
1348,7866,0


In [0]:
resualt=pd.merge(resualt,z,how='inner',on='adId')
#resualt

In [0]:
resualt

,displayId,adId,click_prediction_x,clicked_times_per_ad,click_prediction_y
0,1328976,535,1,17501,8729
1,1328920,535,1,17501,8729
2,1328906,535,0,17501,8729
3,1328893,535,1,17501,8729
4,1328889,535,1,17501,8729
...,...,...,...,...,...
1199043,249134,7865,1,3,1
1199044,248760,7865,0,3,1
1199045,247807,7865,0,3,1
1199046,247209,7867,1,1,1


In [0]:
#resualt['Percentage']=(resualt['click_prediction_y']/resualt['clicked_times_per_ad'])*100
#resualt

In [0]:
resualt=resualt.sort_values(by=['displayId','click_prediction_y'], ascending=False)

In [0]:
resualt

,displayId,adId,click_prediction_x,clicked_times_per_ad,click_prediction_y
0,1328976,535,1,17501,8729
17501,1328976,7182,0,11720,4732
29221,1328976,7174,1,4335,2589
33556,1328976,7167,0,944,798
34500,1328957,6515,0,41874,3672
...,...,...,...,...,...
1167968,151938,123,0,424,146
1035553,151938,7454,0,317,100
1111040,151650,7585,0,5941,1307
1087560,151650,6690,1,1851,658


In [0]:
resualt['Rank']=resualt.groupby(['displayId'])["click_prediction_y"].rank(ascending=False,method='first')
resualt

,displayId,adId,click_prediction_x,clicked_times_per_ad,click_prediction_y,Rank
0,1328976,535,1,17501,8729,1.0
17501,1328976,7182,0,11720,4732,2.0
29221,1328976,7174,1,4335,2589,3.0
33556,1328976,7167,0,944,798,4.0
34500,1328957,6515,0,41874,3672,1.0
...,...,...,...,...,...,...
1167968,151938,123,0,424,146,5.0
1035553,151938,7454,0,317,100,6.0
1111040,151650,7585,0,5941,1307,1.0
1087560,151650,6690,1,1851,658,2.0


In [0]:
#Resualt_MRSH_team=resualt[['displayId','adId','Rank']]

In [0]:
#Resualt_MRSH_team.to_csv('Resualt_MRSH_team.csv',index=False)

In [0]:
#res=pd.read_csv('/content/Resualt_MRSH_team.csv')

In [0]:
#df.to_csv('Preprocessed_X_Train.csv',index=False)

In [0]:
#resualt.reset_index(inplace=False)
#resualt=resualt.sort_values(by=['displayId','clicked_times_per_ad'], ascending=False)
#resualt['Rank']=resualt.groupby(['displayId'])["clicked_times_per_ad"].rank(ascending=False)

In [0]:
resualt=resualt.drop(columns=['click_prediction_x','clicked_times_per_ad','click_prediction_y'])
resualt['Rank']=resualt['Rank'].astype(int)
resualt

,displayId,adId,Rank
0,1328976,535,1
17501,1328976,7182,2
29221,1328976,7174,3
33556,1328976,7167,4
34500,1328957,6515,1
...,...,...,...
1167968,151938,123,5
1035553,151938,7454,6
1111040,151650,7585,1
1087560,151650,6690,2


In [0]:
x=resualt[resualt['displayId']==250719]
x

,displayId,adId,Rank
453163,250719,48,1
1039430,250719,7455,2
657839,250719,7224,3
769679,250719,6394,4
490366,250719,42,5
1155517,250719,7709,6
1182992,250719,7789,7
1155819,250719,7710,8


In [0]:
resualt.to_csv('Resualt_MRSH_team_v2.4.csv',index=False)